
# Sensitivities: Bump vs AAD 

This dashboard demonstrates the accuracy and performance of AAD sensitivities for a range of complex products
- European Option
- Barrier Option
- Accumulator
- Asian Basket Option
- TaRF

all represented as scripted trades.

We are running ORE twice, with differences in the pricinengine.xml file:
- setting UseAD to false for the bump run and
- setting UseAD to true for AAD.

## Run ORE, Bump & Reval Sensitivities, Single-Threaded

In [1]:
from ORE import *
import sys, time, math
sys.path.append('..')
import utilities

In [2]:
params = Parameters()
params.fromFile("Input/ore.xml")
ore = OREApp(params)
ore.run()
utilities.checkErrorsAndRunTime(ore)
#utilities.writeList(ore.getReportNames())

Run time: 2.76 sec
Errors: 0


## Run ORE using AAD

Check the following is set in **pricingengine_ad.xml**
- **UseAD** is set to true

Check the following is set in **sensitivity.xml**
- **ComputeGamma** is set to false: otherwise AD is deactivated, because it does not support gamma and cross gamma calculation so far


In [3]:
params_ad = Parameters()
params_ad.fromFile("Input/ore_ad.xml")
ore_ad = OREApp(params_ad)
ore_ad.run()
utilities.checkErrorsAndRunTime(ore_ad)
#utilities.writeList(ore_ad.getReportNames())

Run time: 0.80 sec
Errors: 0


In [4]:
# Compare Sensitivities
utilities.match_sensi_reports(ore, ore_ad, "DeltaBump", "DeltaAD", False)

Trade                     Factor                                          Currency  DeltaBump    DeltaAD       Diff
3:DIGITAL                 DiscountCurve/EUR/4/1Y                               EUR     -25.37     -25.37       0.00
3:DIGITAL                 DiscountCurve/EUR/5/2Y                               EUR     -43.21     -43.21       0.00
3:DIGITAL                 DiscountCurve/USD/4/1Y                               EUR      22.26      22.26       0.00
3:DIGITAL                 DiscountCurve/USD/5/2Y                               EUR      37.91      37.91       0.00
3:DIGITAL                 FXSpot/USDEUR/0/spot                                 EUR    -411.26    -411.26       0.00
3:DIGITAL                 FXVolatility/EURUSD/88/1Y/-0.5                       EUR      -0.42      -0.42       0.00
3:DIGITAL                 FXVolatility/EURUSD/89/2Y/-0.5                       EUR      -0.37      -0.37       0.00
3:DIGITAL                 FXVolatility/EURUSD/100/1Y/-0.25              

### Tasks

Run the notebook. You should see significant differences in the "Scripted Digital", TARF with Knockout and TARF with Leverages.

**1. Change sensitivity.xml as follows and then compare bump and AD sensitivities again**
- reduce discount curve shift size from 1e-4 to 1e-7
- reduce fx spot shift size from 1e-3 to 1e-6
- set the fx spot sift scheme from Central to Forward
- reduce the fx volatility shift size from 1e-3 to 1e-5

Can you explain the "sensitivity" of bump sensitivities ?


**2. Then check the impact of epsilon** 

The default is gradient calculation with epsilon = 0.2. 
- Now modify the source code to expose the epsilon parameters (one affects the payoff, the other one the gradient, we are interested in modifying the second) such that we can amend them in the pricingengine_ad.xml.
- Compile
- Then rerun the sensitivity analysis with AAD, but now for epsilon = 0. Call "python runad.py" to do this, because
you haven't updated the ORE Python module (yet). runad.py uses the command line executable.
- How did sensitivities change? Focus on the epsilon impact on the FX spot sensitivity in Scripted DIGITAL, TARF 2 and TARF 3. 

We will discuss in the final week's live session.
